In [67]:
import numpy as np
from matplotlib import pyplot as plt
import cv2

## Capture


In [74]:
# define a video capture object
vid = cv2.VideoCapture(0)
  
while(True):
      
    # Capture the video frame
    # by frame
    ret, frame = vid.read()
  
    # Display the resulting frame
    cv2.imshow('frame', frame)
      
    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  
# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()

## Thresholding

In [75]:
img_path = 'hand.jpg'
# Read the image using openCV
original_img = cv2.imread(img_path)
original_img = frame
img = cv2.cvtColor(original_img, cv2.COLOR_BGR2HSV)
lower = np.array([0,40,40], dtype=np.uint8)
upper = np.array([20,255,255], dtype=np.uint8)
skinRegionHSV = cv2.inRange(img, lower, upper)
blurred = cv2.blur(skinRegionHSV, (2,2))
ret,thresh = cv2.threshold(blurred,0,255,cv2.THRESH_BINARY)

cv2.imshow('thresh',thresh)
cv2.waitKey(0) # Wait for user input, press any key
cv2.destroyAllWindows()


## Contours

In [76]:
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = max(contours, key=lambda x: cv2.contourArea(x))
cv2.drawContours(img, [contours], -1, (255,255,0), 2)
cv2.imshow("contours", img)

cv2.waitKey(0) # Wait for user input, press any key
cv2.destroyAllWindows()

## Convex Hull

In [77]:
hull = cv2.convexHull(contours)
cv2.drawContours(img, [hull], -1, (0, 255, 255), 2)
cv2.imshow("hull", img)

cv2.waitKey(0) # Wait for user input, press any key
cv2.destroyAllWindows()

## ROI Extraction

In [78]:
min_x, max_x, min_y, max_y = min(hull[:,:,0])[0], max(hull[:,:,0])[0],min(hull[:,:,1])[0],max(hull[:,:,1])[0]
print(min_x, max_x, min_y, max_y)

42 276 142 479


In [40]:
delta = 10
roi = img[max(0,min_y-delta):min(max_y+delta, img.shape[0]), max(0,min_x-delta):min(max_x+delta,img.shape[1])]
cv2.imshow("roi", roi)

cv2.waitKey(0) # Wait for user input, press any key
cv2.destroyAllWindows()

# Complete Function to be used

In [79]:
def segment_hand(frame, delta=10):
    """"
    This function extract the area of an image which contains a hand.
    @ Parameters:
        frame: the frame (image) to extract the hand region from
        delta: relaxation parameter (0:20) default value = 10
    @ Returns:
        roi: The region of interest which includes the hand
    """
    # Skin color limits
    lower_limit = np.array([0,40,40], dtype=np.uint8)
    upper_limit = np.array([20,255,255], dtype=np.uint8)
    # Convert color to HSV domain
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # Decide the skin region
    skin_region = cv2.inRange(img, lower_limit, upper_limit)
    # Apply thresholding segmentation
    ret,thresholded = cv2.threshold(skin_region,0,255,cv2.THRESH_BINARY)
    # Find contours around the thresholded regions
    contours, hierarchy = cv2.findContours(thresholded, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = max(contours, key=lambda x: cv2.contourArea(x))
    # Apply convex hull algorithm
    convex_hull = cv2.convexHull(contours)
    # Decide the area of contours (with delta relaxation)
    min_x, max_x, min_y, max_y = min(convex_hull[:,:,0])[0], max(convex_hull[:,:,0])[0],min(convex_hull[:,:,1])[0],max(convex_hull[:,:,1])[0]
    roi = [max(0,min_y-delta), min(max_y+delta, frame.shape[0]), max(0,min_x-delta), min(max_x+delta,frame.shape[1])]
    return roi

In [80]:
roi_ = segment_hand(frame,10)
roi_

[132, 480, 32, 286]

In [81]:
roi_ = segment_hand(frame,10)
cv2.imshow("roi", frame[roi_[0]:roi_[1], roi_[2]:roi_[3]])

cv2.waitKey(0) # Wait for userinput, press any key
cv2.destroyAllWindows()